*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you will create a book recommendation algorithm using **K-Nearest Neighbors**.

You will use the [Book-Crossings dataset](http://www2.informatik.uni-freiburg.de/~cziegler/BX/). This dataset contains 1.1 million ratings (scale of 1-10) of 270,000 books by 90,000 users. 

After importing and cleaning the data, use `NearestNeighbors` from `sklearn.neighbors` to develop a model that shows books that are similar to a given book. The Nearest Neighbors algorithm measures distance to determine the “closeness” of instances.

Create a function named `get_recommends` that takes a book title (from the dataset) as an argument and returns a list of 5 similar books with their distances from the book argument.

This code:

`get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")`

should return:

```
[
  'The Queen of the Damned (Vampire Chronicles (Paperback))',
  [
    ['Catch 22', 0.793983519077301], 
    ['The Witching Hour (Lives of the Mayfair Witches)', 0.7448656558990479], 
    ['Interview with the Vampire', 0.7345068454742432],
    ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.5376338362693787],
    ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.5178412199020386]
  ]
]
```

Notice that the data returned from `get_recommends()` is a list. The first element in the list is the book title passed in to the function. The second element in the list is a list of five more lists. Each of the five lists contains a recommended book and the distance from the recommended book to the book passed in to the function.

If you graph the dataset (optional), you will notice that most books are not rated frequently. To ensure statistical significance, remove from the dataset users with less than 200 ratings and books with less than 100 ratings.

The first three cells import libraries you may need and the data to use. The final cell is for testing. Write all your code in between those cells.

In [4]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [5]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2020-09-02 20:19:42--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 157.230.45.115, 157.230.37.202, 2400:6180:0:d1::684:6001, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|157.230.45.115|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M   348KB/s    in 76s     

2020-09-02 20:20:59 (335 KB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [6]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [7]:
# add your code here - consider creating a new cell for each section of code

In [8]:
df_merge = pd.merge(df_ratings, df_books, on='isbn')
df_books_cnt = pd.DataFrame(df_merge.groupby('isbn').size(), columns=['count'])
popular_books = list(set(df_books_cnt.query('count >= 100').index))
books_filter = df_merge.isbn.isin(popular_books).values

df_users_cnt = pd.DataFrame(df_merge.groupby('user').size(), columns=['count'])
active_users = list(set(df_users_cnt.query('count >= 200').index))
users_filter = df_merge.user.isin(active_users).values

df_merge_filtered = df_merge[books_filter & users_filter]
df_merge_filtered.head(10)


# df_test = df_merge.groupby('isbn', as_index=False).count()
# df_test = df_test.sort_values('rating', ascending=False)
# plt.plot(df_test['isbn'][:1000], df_test['rating'][:1000])

,user,isbn,rating,title,author
63,278418,0446520802,0.0,The Notebook,Nicholas Sparks
65,3363,0446520802,0.0,The Notebook,Nicholas Sparks
66,7158,0446520802,10.0,The Notebook,Nicholas Sparks
69,11676,0446520802,10.0,The Notebook,Nicholas Sparks
74,23768,0446520802,6.0,The Notebook,Nicholas Sparks
77,27617,0446520802,9.0,The Notebook,Nicholas Sparks
78,28204,0446520802,0.0,The Notebook,Nicholas Sparks
79,29855,0446520802,0.0,The Notebook,Nicholas Sparks
81,30711,0446520802,6.0,The Notebook,Nicholas Sparks
82,32440,0446520802,0.0,The Notebook,Nicholas Sparks


In [9]:
df = df_merge_filtered[['user', 'rating', 'title']]
df = df.drop_duplicates(['title', 'user'])
df.head()


,user,rating,title
63,278418,0.0,The Notebook
65,3363,0.0,The Notebook
66,7158,10.0,The Notebook
69,11676,10.0,The Notebook
74,23768,6.0,The Notebook


In [10]:
books_pivot = df.pivot(index='title', columns='user', values='rating').fillna(0)
books_pivot_sparse = csr_matrix(books_pivot.values)

In [11]:
knn = NearestNeighbors(n_neighbors=5, algorithm='brute', metric='cosine', n_jobs=-1)
model_knn = knn.fit(books_pivot_sparse)

In [12]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  distance, indice = model_knn.kneighbors(books_pivot[books_pivot.index== book], n_neighbors=5)
  # distance, indice = model_knn.kneighbors(books_pivot.iloc[books_pivot.index== book].values.reshape(1,-1),n_neighbors=6)
  recommended_books = []
  tmp = []
  for i in range(0, len(distance.flatten())):
    if  i == 0:
      recommended_books.append(books_pivot[books_pivot.index== book].index[0])
    else:
      tmp.insert(0, [books_pivot.index[indice.flatten()[i]], distance.flatten()[i]])
  recommended_books.append(tmp)
  return recommended_books

get_recommends('The Queen of the Damned (Vampire Chronicles (Paperback))')

['The Queen of the Damned (Vampire Chronicles (Paperback))',
 [['The Witching Hour (Lives of the Mayfair Witches)', 0.7362787],
  ['Interview with the Vampire', 0.73255134],
  ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.5298544],
  ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.5145134]]]

Use the cell below to test your function. The `test_book_recommendation()` function will inform you if you passed the challenge or need to keep trying.

In [13]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You havn't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [['The Weight of Water', 0.7678344], ['The Surgeon', 0.76690507], ['I Know This Much Is True', 0.764642], ['The Lovely Bones: A Novel', 0.7158567]]]
You passed the challenge! 🎉🎉🎉🎉🎉
